In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('../../../../../Downloads/preprocessed_data.csv')

In [3]:
df

,age,gender,cholesterol,gluc,smoke,alco,active,cardio,male,female,bmi,bpc
0,18393,2,1,1,0,0,1,0,1,0,21.967120,3
1,20228,1,3,1,0,0,1,1,0,1,34.927679,4
2,18857,1,3,1,0,0,0,1,0,1,23.507805,3
3,17623,2,1,1,0,0,1,1,1,0,28.710479,4
4,17474,1,1,1,0,0,0,0,0,1,23.011177,1
...,...,...,...,...,...,...,...,...,...,...,...,...
68656,19240,2,1,1,1,0,1,0,1,0,26.927438,3
68657,22601,1,2,2,0,0,1,1,0,1,50.472681,4
68658,19066,2,3,1,0,1,0,1,1,0,31.353579,5
68659,22431,1,1,2,0,0,0,1,0,1,27.099251,3


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [5]:
y=df['cardio']

In [6]:
y

0        0
1        1
2        1
3        1
4        0
        ..
68656    0
68657    1
68658    1
68659    1
68660    0
Name: cardio, Length: 68661, dtype: int64

In [7]:
df.drop('cardio',axis=1,inplace=True)

In [8]:
X_train,X_test,y_train,y_test=train_test_split(df,y,random_state=3)

In [9]:
X_train.shape

(51495, 11)

In [10]:
X_test.shape

(17166, 11)

In [11]:
y_train.shape

(51495,)

In [12]:
forest_clf=RandomForestClassifier(n_jobs=3)

In [13]:
forest_clf

RandomForestClassifier(n_jobs=3)

In [14]:
from sklearn.model_selection import cross_val_score

In [15]:
forest_clf.fit(X_train,y_train)

RandomForestClassifier(n_jobs=3)

In [16]:
y_pred=forest_clf.predict(X_test)

In [17]:
y_pred

array([1, 1, 0, ..., 0, 0, 1], dtype=int64)

In [18]:
from sklearn.metrics import accuracy_score

In [19]:
accuracy_score(y_test,y_pred)

0.682744960969358

In [20]:
train_score=cross_val_score(forest_clf,X_train,y_train,scoring='accuracy')

In [21]:
train_score

array([0.68336732, 0.68725119, 0.69463055, 0.68346441, 0.68890184])

In [22]:
np.mean(train_score)

0.6875230604913098

In [23]:
f1=cross_val_score(forest_clf,X_train,y_train,scoring='f1')

In [24]:
np.mean(f1)

0.6813495368996272

In [25]:
from sklearn.linear_model import LogisticRegression

In [26]:
logistic_classifer=LogisticRegression(solver='liblinear')

In [27]:
train_logistic=cross_val_score(logistic_classifer,X_train,y_train,scoring='accuracy')

In [28]:
np.mean(train_logistic)

0.7115448101757453

In [29]:
from sklearn.ensemble import AdaBoostClassifier
adaboost_clf=AdaBoostClassifier(algorithm='SAMME.R',random_state=42)

In [30]:
train_ada=cross_val_score(adaboost_clf,X_train,y_train,scoring='accuracy',n_jobs=3)

In [31]:
np.mean(train_ada)

0.7239732012816779

In [32]:
from sklearn.neural_network import MLPClassifier

In [33]:
mlp_clf=MLPClassifier(hidden_layer_sizes=(128,256,128),max_iter=300,random_state=42)

In [34]:
mlp_clf.fit(X_train,y_train)

MLPClassifier(hidden_layer_sizes=(128, 256, 128), max_iter=300, random_state=42)

In [36]:
y_pred_mlp=mlp_clf.predict(X_test)

In [37]:
accuracy_score(y_test,y_pred_mlp)

0.6147034836304323

In [39]:
from sklearn.ensemble import VotingClassifier
v_clf=VotingClassifier([('lr',logistic_classifer),('rf',forest_clf),('ada',adaboost_clf)],voting='soft')

In [40]:
v_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(solver='liblinear')),
                             ('rf', RandomForestClassifier(n_jobs=3)),
                             ('ada', AdaBoostClassifier(random_state=42))],
                 voting='soft')

In [41]:
y_pred_voting=v_clf.predict(X_test)

In [42]:
accuracy_score(y_test,y_pred_voting)

0.7045904695327974

In [43]:
from sklearn.metrics import f1_score

In [44]:
f1_score(y_test,y_pred_voting)

0.7010199870290666

In [49]:
v_clf.estimators_

[LogisticRegression(solver='liblinear'),
 RandomForestClassifier(n_jobs=3),
 AdaBoostClassifier(random_state=42)]

In [50]:
y_pred_logistic=v_clf.estimators_[0].predict(X_test)
accuracy_score(y_test,y_pred_logistic),f1_score(y_test,y_pred_logistic)

(0.7084352790399627, 0.7069157346138081)

In [51]:
y_pred_forest=v_clf.estimators_[1].predict(X_test)
accuracy_score(y_test,y_pred_forest),f1_score(y_test,y_pred_forest)

(0.6833857625538856, 0.6791428065411182)

In [53]:
y_pred_ada=v_clf.estimators_[2].predict(X_test)
accuracy_score(y_test,y_pred_ada),f1_score(y_test,y_pred_ada)

(0.7154258417802634, 0.6918564309594399)

In [55]:
accuracy_score(y_test,y_pred_voting),f1_score(y_test,y_pred_voting)

(0.7045904695327974, 0.7010199870290666)

In [ ]:
from sklearn.neighbors import kneighbors_graph